In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import sklearn.model_selection as model_selection
from sklearn.preprocessing import LabelEncoder
import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
%matplotlib inline

warnings.filterwarnings('ignore')

In [2]:
data=pd.read_csv('1805084__.csv')

In [3]:
data['clear_date']=pd.to_datetime(data['clear_date'].astype(str),format='%Y-%m-%d')
data['posting_date']=pd.to_datetime(data['posting_date'],format='%Y-%m-%d')
data['document_create_date']=pd.to_datetime(data['document_create_date'].astype(str),format='%Y-%m-%d')
data['document_create_date.1']=pd.to_datetime(data['document_create_date.1'].astype(str),format='%Y-%m-%d')
data['baseline_create_date']=pd.to_datetime(data['baseline_create_date'].astype(str),format='%Y%m%d')
data['due_in_date']=pd.to_datetime(data['due_in_date'].astype(str),format='%Y%m%d')

In [4]:

data['Delay']=((data['clear_date']-data['due_in_date']).dt.days)

In [5]:
data['Delay'].max()

223.0

In [6]:
train_set = pd.DataFrame(data[(data['clear_date'].notnull())&(data['isOpen']==0)])

test_set = pd.DataFrame(data[(data['clear_date'].isnull())&(data['isOpen']==1)])

In [7]:
train_set.shape,test_set.shape

((46587, 20), (3413, 20))

In [8]:
train_set.sort_values(by='document_create_date.1',ascending=True).shape

(46587, 20)

In [9]:
train_data,val=train_test_split(train_set,test_size=0.3,shuffle=False,stratify=None)

In [10]:
train_data.shape,val.shape

((32610, 20), (13977, 20))

In [11]:
train_data.sort_values(by='document_create_date.1',ascending=True).shape

(32610, 20)

train_data.nunique()

In [12]:
train_data=train_data[train_data['document_create_date.1']<=train_data['posting_date']]

In [13]:
train_data.shape

(32610, 20)

In [14]:
train_data=train_data[train_data['document_create_date.1']<=train_data['baseline_create_date']]
train_data=train_data[train_data['document_create_date.1']<=train_data['due_in_date']]
train_data=train_data[train_data['document_create_date.1']<=train_data['clear_date']]
train_data=train_data[train_data['posting_date']<=train_data['baseline_create_date']]
train_data=train_data[train_data['posting_date']<=train_data['due_in_date']]
train_data=train_data[train_data['posting_date']<=train_data['clear_date']]
train_data=train_data[train_data['baseline_create_date']<=train_data['due_in_date']]
train_data=train_data[train_data['baseline_create_date']<=train_data['clear_date']]

In [15]:
train_data.shape

(31193, 20)

In [16]:
train_data=train_data[((train_data['due_in_date']<=train_data['clear_date'])&(train_data['Delay']>=0))|((train_data['due_in_date']>train_data['clear_date'])&(train_data['Delay']<=0))]

In [17]:
train_data.shape

(31193, 20)

In [18]:
train_data.dtypes

business_code                     object
cust_number                       object
name_customer                     object
clear_date                datetime64[ns]
buisness_year                    float64
doc_id                           float64
posting_date              datetime64[ns]
document_create_date      datetime64[ns]
document_create_date.1    datetime64[ns]
due_in_date               datetime64[ns]
invoice_currency                  object
document type                     object
posting_id                       float64
area_business                    float64
total_open_amount                float64
baseline_create_date      datetime64[ns]
cust_payment_terms                object
invoice_id                       float64
isOpen                             int64
Delay                            float64
dtype: object

In [19]:
train_data[train_data.isnull().any(axis=1)]

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,document type,posting_id,area_business,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen,Delay
0,U001,0200705742,DOT corp,2020-03-16,2020.0,1.930523e+09,2020-02-12,2020-02-13,2020-02-12,2020-03-17,USD,RV,1.0,NaN,7786.44,2020-02-12,NAAW,1.930523e+09,0,-1.0
1,CA02,0140104423,METRO systems,2020-02-17,2020.0,2.960613e+09,2020-02-07,2020-02-07,2020-02-07,2020-02-21,CAD,RV,1.0,NaN,36987.58,2020-02-11,CA10,2.960613e+09,0,-4.0
2,U001,0200920735,ALBERT us,2019-08-06,2019.0,1.929645e+09,2019-07-22,2019-07-20,2019-07-22,2019-08-06,USD,RV,1.0,NaN,71807.69,2019-07-22,NAA8,1.929645e+09,0,0.0
3,U001,0200701314,MBM associates,2019-11-26,2019.0,1.930124e+09,2019-11-08,2019-11-07,2019-11-08,2019-11-23,USD,RV,1.0,NaN,2575.46,2019-11-08,NAA8,1.930124e+09,0,3.0
4,U001,0200769623,WAL-MAR in,2019-02-01,2019.0,1.928649e+09,2019-01-20,2019-01-19,2019-01-20,2019-02-04,USD,RV,1.0,NaN,15576.52,2019-01-20,NAH4,1.928649e+09,0,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32605,U001,0200704858,WAKE,2019-07-10,2019.0,1.929535e+09,2019-06-25,2019-06-24,2019-06-25,2019-07-10,USD,RV,1.0,NaN,11300.18,2019-06-25,NAA8,1.929535e+09,0,0.0
32606,U001,CCU013,KRAFT F corporation,2019-05-03,2019.0,1.929039e+09,2019-03-30,2019-03-28,2019-03-30,2019-03-30,USD,RV,1.0,NaN,28341.16,2019-03-30,NAX2,1.929039e+09,0,34.0
32607,U001,0200769623,WAL-MAR corp,2019-09-04,2019.0,1.929760e+09,2019-08-23,2019-08-18,2019-08-23,2019-09-07,USD,RV,1.0,NaN,22310.99,2019-08-23,NAH4,1.929760e+09,0,-3.0
32608,U001,0200794332,COST associates,2019-12-19,2019.0,1.930211e+09,2019-12-05,2019-12-03,2019-12-05,2019-12-20,USD,RV,1.0,NaN,15791.62,2019-12-05,NAAX,1.930211e+09,0,-1.0


In [20]:
train_data['business_code'].value_counts()

U001    28144
CA02     2521
U013      402
U002      119
U005        5
U007        2
Name: business_code, dtype: int64

In [21]:
train_data[train_data['business_code']=='U007']

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,document type,posting_id,area_business,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen,Delay
17929,U007,0100024767,GREEN systems,2019-06-12,2019.0,1.928626e+09,2019-01-15,2019-01-15,2019-01-15,2019-02-19,USD,RV,1.0,NaN,716.0,2019-01-15,NAG2,1.928626e+09,0,113.0
30984,U007,0100024767,GREEN systems,2019-06-05,2019.0,1.928932e+09,2019-03-08,2019-03-08,2019-03-08,2019-04-12,USD,RV,1.0,NaN,1443.2,2019-03-08,NAG2,1.928932e+09,0,54.0


In [22]:
train_data1=pd.get_dummies(train_data,columns=["business_code"])

In [23]:
#train_data1.info()
train_data1.shape

(31193, 25)

In [24]:
train_data1['business_code_U007'].value_counts()

0    31191
1        2
Name: business_code_U007, dtype: int64

In [25]:
train_data1

,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,...,cust_payment_terms,invoice_id,isOpen,Delay,business_code_CA02,business_code_U001,business_code_U002,business_code_U005,business_code_U007,business_code_U013
0,0200705742,DOT corp,2020-03-16,2020.0,1.930523e+09,2020-02-12,2020-02-13,2020-02-12,2020-03-17,USD,...,NAAW,1.930523e+09,0,-1.0,0,1,0,0,0,0
1,0140104423,METRO systems,2020-02-17,2020.0,2.960613e+09,2020-02-07,2020-02-07,2020-02-07,2020-02-21,CAD,...,CA10,2.960613e+09,0,-4.0,1,0,0,0,0,0
2,0200920735,ALBERT us,2019-08-06,2019.0,1.929645e+09,2019-07-22,2019-07-20,2019-07-22,2019-08-06,USD,...,NAA8,1.929645e+09,0,0.0,0,1,0,0,0,0
3,0200701314,MBM associates,2019-11-26,2019.0,1.930124e+09,2019-11-08,2019-11-07,2019-11-08,2019-11-23,USD,...,NAA8,1.930124e+09,0,3.0,0,1,0,0,0,0
4,0200769623,WAL-MAR in,2019-02-01,2019.0,1.928649e+09,2019-01-20,2019-01-19,2019-01-20,2019-02-04,USD,...,NAH4,1.928649e+09,0,-3.0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32605,0200704858,WAKE,2019-07-10,2019.0,1.929535e+09,2019-06-25,2019-06-24,2019-06-25,2019-07-10,USD,...,NAA8,1.929535e+09,0,0.0,0,1,0,0,0,0
32606,CCU013,KRAFT F corporation,2019-05-03,2019.0,1.929039e+09,2019-03-30,2019-03-28,2019-03-30,2019-03-30,USD,...,NAX2,1.929039e+09,0,34.0,0,1,0,0,0,0
32607,0200769623,WAL-MAR corp,2019-09-04,2019.0,1.929760e+09,2019-08-23,2019-08-18,2019-08-23,2019-09-07,USD,...,NAH4,1.929760e+09,0,-3.0,0,1,0,0,0,0
32608,0200794332,COST associates,2019-12-19,2019.0,1.930211e+09,2019-12-05,2019-12-03,2019-12-05,2019-12-20,USD,...,NAAX,1.930211e+09,0,-1.0,0,1,0,0,0,0


In [26]:
train_data1['document_create_date.1.year']=train_data1['document_create_date.1'].dt.year
train_data1['document_create_date.1.month']=train_data1['document_create_date.1'].dt.month
train_data1['document_create_date.1.Day']=train_data1['document_create_date.1'].dt.day

In [27]:
train_data1.shape

(31193, 28)

In [28]:
train_data1['due_in_date.year']=train_data1['due_in_date'].dt.year
train_data1['due_in_date.month']=train_data1['due_in_date'].dt.month
train_data1['due_in_date.Day']=train_data1['due_in_date'].dt.day

In [29]:
train_data1.shape

(31193, 31)

In [30]:
train_data1['invoice_currency'].value_counts()

USD    28671
CAD     2522
Name: invoice_currency, dtype: int64

In [31]:
train_data1["invoice_currency"]=train_data1["invoice_currency"].astype('category')

In [32]:
train_data1["invoice_currency_labelencoded"]=train_data1["invoice_currency"].cat.codes

In [33]:
train_data1.shape
#train_data1.head()

(31193, 32)

In [34]:
train_data1["invoice_currency_labelencoded"].value_counts()

1    28671
0     2522
Name: invoice_currency_labelencoded, dtype: int64

In [35]:
train_data1['cust_payment_terms'].value_counts()

NAA8    13110
NAH4     8655
CA10     2456
NAC6     1122
NAAX      772
        ...  
NATX        1
NACE        1
CAX2        1
BR12        1
NAWK        1
Name: cust_payment_terms, Length: 66, dtype: int64

In [36]:
train_data1_mean=train_data1.groupby('cust_payment_terms').mean()
#train_data1_mean

In [37]:
#train_data1_mean
train_data1_mean.shape

(66, 21)

In [38]:
train_data1.shape

(31193, 32)

In [39]:
train_data1['cust_payment_terms'].describe()

count     31193
unique       66
top        NAA8
freq      13110
Name: cust_payment_terms, dtype: object

In [40]:
train_data1[['cust_payment_terms','Delay']]

,cust_payment_terms,Delay
0,NAAW,-1.0
1,CA10,-4.0
2,NAA8,0.0
3,NAA8,3.0
4,NAH4,-3.0
...,...,...
32605,NAA8,0.0
32606,NAX2,34.0
32607,NAH4,-3.0
32608,NAAX,-1.0


In [41]:
x=train_data1.groupby(['cust_payment_terms'])['Delay'].mean().to_dict()

In [42]:
#x

In [43]:
train_data1['cust_payment_terms_tencoded']=train_data1['cust_payment_terms'].map(x)

In [44]:
train_data1.shape

(31193, 33)

In [45]:


train_data1['Delay']

0        -1.0
1        -4.0
2         0.0
3         3.0
4        -3.0
         ... 
32605     0.0
32606    34.0
32607    -3.0
32608    -1.0
32609     0.0
Name: Delay, Length: 31193, dtype: float64

In [46]:
train_data1.drop(['cust_number','name_customer','invoice_currency','document type','cust_payment_terms','clear_date','posting_date','document_create_date','document_create_date.1','due_in_date','baseline_create_date'],axis=1,inplace=True)

In [47]:
train_data1.shape

(31193, 22)

In [48]:
constant_filter1=VarianceThreshold(threshold=0)
constant_filter1.fit(train_data1)

VarianceThreshold(threshold=0)

In [49]:
constant_filter1.get_support().sum()

19

In [50]:
constant_columns = [column for column in train_data1.columns
                    if column not in train_data1.columns[constant_filter1.get_support()]]

print((constant_columns))

['posting_id', 'area_business', 'isOpen']


In [51]:
quasi_constant_filter1=VarianceThreshold(threshold=0.01)

In [52]:
quasi_constant_filter1.fit(train_data1)

VarianceThreshold(threshold=0.01)

In [53]:
quasi_constant_filter1.get_support().sum()

16

In [54]:
constant_columns = [column for column in train_data1.columns
                    if column not in train_data1.columns[quasi_constant_filter1.get_support()]]

print((constant_columns))

['posting_id', 'area_business', 'isOpen', 'business_code_U002', 'business_code_U005', 'business_code_U007']


In [55]:
train_data1.drop(['posting_id','area_business','isOpen','business_code_U002','business_code_U005', 'business_code_U007'],axis=1,inplace=True)

In [56]:
X_train_T=train_data1.T

In [57]:
X_train_T.shape

(16, 31193)

In [58]:
X_train_T=pd.DataFrame(X_train_T)

In [59]:
X_train_T.duplicated().sum()

1

In [60]:
duplicated_feature1=X_train_T.duplicated()
duplicated_feature1

buisness_year                    False
doc_id                           False
total_open_amount                False
invoice_id                        True
Delay                            False
business_code_CA02               False
business_code_U001               False
business_code_U013               False
document_create_date.1.year      False
document_create_date.1.month     False
document_create_date.1.Day       False
due_in_date.year                 False
due_in_date.month                False
due_in_date.Day                  False
invoice_currency_labelencoded    False
cust_payment_terms_tencoded      False
dtype: bool

In [61]:
features_to_keep1=[not index for index in duplicated_feature1]

In [62]:
X_train_unique=X_train_T[features_to_keep1].T

In [63]:
X_train_unique.shape

(31193, 15)

In [64]:
train_data1=X_train_unique

In [65]:
train_data1

,buisness_year,doc_id,total_open_amount,Delay,business_code_CA02,business_code_U001,business_code_U013,document_create_date.1.year,document_create_date.1.month,document_create_date.1.Day,due_in_date.year,due_in_date.month,due_in_date.Day,invoice_currency_labelencoded,cust_payment_terms_tencoded
0,2020.0,1.930523e+09,7786.44,-1.0,0.0,1.0,0.0,2020.0,2.0,12.0,2020.0,3.0,17.0,1.0,-15.150000
1,2020.0,2.960613e+09,36987.58,-4.0,1.0,0.0,0.0,2020.0,2.0,7.0,2020.0,2.0,21.0,0.0,5.545603
2,2019.0,1.929645e+09,71807.69,0.0,0.0,1.0,0.0,2019.0,7.0,22.0,2019.0,8.0,6.0,1.0,1.329291
3,2019.0,1.930124e+09,2575.46,3.0,0.0,1.0,0.0,2019.0,11.0,8.0,2019.0,11.0,23.0,1.0,1.329291
4,2019.0,1.928649e+09,15576.52,-3.0,0.0,1.0,0.0,2019.0,1.0,20.0,2019.0,2.0,4.0,1.0,-1.887464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32605,2019.0,1.929535e+09,11300.18,0.0,0.0,1.0,0.0,2019.0,6.0,25.0,2019.0,7.0,10.0,1.0,1.329291
32606,2019.0,1.929039e+09,28341.16,34.0,0.0,1.0,0.0,2019.0,3.0,30.0,2019.0,3.0,30.0,1.0,43.228216
32607,2019.0,1.929760e+09,22310.99,-3.0,0.0,1.0,0.0,2019.0,8.0,23.0,2019.0,9.0,7.0,1.0,-1.887464
32608,2019.0,1.930211e+09,15791.62,-1.0,0.0,1.0,0.0,2019.0,12.0,5.0,2019.0,12.0,20.0,1.0,0.527202


In [66]:
train_data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31193 entries, 0 to 32609
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   buisness_year                  31193 non-null  float64
 1   doc_id                         31193 non-null  float64
 2   total_open_amount              31193 non-null  float64
 3   Delay                          31193 non-null  float64
 4   business_code_CA02             31193 non-null  float64
 5   business_code_U001             31193 non-null  float64
 6   business_code_U013             31193 non-null  float64
 7   document_create_date.1.year    31193 non-null  float64
 8   document_create_date.1.month   31193 non-null  float64
 9   document_create_date.1.Day     31193 non-null  float64
 10  due_in_date.year               31193 non-null  float64
 11  due_in_date.month              31193 non-null  float64
 12  due_in_date.Day                31193 non-null 

In [67]:
val.sort_values(by='document_create_date.1',ascending=True).shape

(13977, 20)

In [68]:
val.nunique()

business_code                 6
cust_number                 955
name_customer              2406
clear_date                  357
buisness_year                 2
doc_id                    13977
posting_date                420
document_create_date        422
document_create_date.1      420
due_in_date                 471
invoice_currency              2
document type                 1
posting_id                    1
area_business                 0
total_open_amount         13215
baseline_create_date        423
cust_payment_terms           59
invoice_id                13977
isOpen                        1
Delay                       146
dtype: int64

In [69]:
val=val[val['document_create_date.1']<=val['posting_date']]

In [70]:
val.shape

(13977, 20)

In [71]:
val=val[val['document_create_date.1']<=val['baseline_create_date']]
val=val[val['document_create_date.1']<=val['due_in_date']]
val=val[val['document_create_date.1']<=val['clear_date']]
val=val[val['posting_date']<=val['baseline_create_date']]
val=val[val['posting_date']<=val['due_in_date']]
val=val[val['posting_date']<=val['clear_date']]
val=val[val['baseline_create_date']<=val['due_in_date']]
val=val[val['baseline_create_date']<=val['clear_date']]

In [72]:
val.shape

(13333, 20)

In [73]:
val=val[((val['due_in_date']<=val['clear_date'])&(val['Delay']>=0))|((val['due_in_date']>val['clear_date'])&(val['Delay']<=0))]

In [74]:
val.shape

(13333, 20)

In [75]:
val.dtypes

business_code                     object
cust_number                       object
name_customer                     object
clear_date                datetime64[ns]
buisness_year                    float64
doc_id                           float64
posting_date              datetime64[ns]
document_create_date      datetime64[ns]
document_create_date.1    datetime64[ns]
due_in_date               datetime64[ns]
invoice_currency                  object
document type                     object
posting_id                       float64
area_business                    float64
total_open_amount                float64
baseline_create_date      datetime64[ns]
cust_payment_terms                object
invoice_id                       float64
isOpen                             int64
Delay                            float64
dtype: object

In [76]:
val[val.isnull().any(axis=1)]

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,document type,posting_id,area_business,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen,Delay
32610,U001,0200769623,WAL-MAR,2020-02-28,2020.0,1.930517e+09,2020-02-12,2020-02-12,2020-02-12,2020-02-27,USD,RV,1.0,NaN,6899.20,2020-02-12,NAH4,1.930517e+09,0,1.0
32611,U001,0200769623,WAL-MAR in,2019-12-09,2019.0,1.930198e+09,2019-11-28,2019-11-27,2019-11-28,2019-12-13,USD,RV,1.0,NaN,1897.50,2019-11-28,NAH4,1.930198e+09,0,-4.0
32612,U001,0200210681,HUB,2019-09-11,2019.0,1.929774e+09,2019-08-23,2019-08-22,2019-08-23,2019-09-12,USD,RV,1.0,NaN,13479.51,2019-08-23,NAD1,1.929774e+09,0,-1.0
32613,U001,0200729942,SA llc,2019-03-18,2019.0,1.928857e+09,2019-02-25,2019-02-25,2019-02-25,2019-03-12,USD,RV,1.0,NaN,9508.41,2019-02-25,NAA8,1.928857e+09,0,6.0
32615,U001,0200726979,BJ'S in,2019-02-19,2019.0,1.928727e+09,2019-02-02,2019-02-02,2019-02-02,2019-02-17,USD,RV,1.0,NaN,1831.09,2019-02-02,NAA8,1.928727e+09,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46582,U001,0200759878,SA,2019-06-04,2019.0,1.929343e+09,2019-05-18,2019-05-18,2019-05-18,2019-06-02,USD,RV,1.0,NaN,39747.01,2019-05-18,NAH4,1.929343e+09,0,2.0
46583,CA02,0140104440,SO co,2019-04-03,2019.0,2.960536e+09,2019-03-17,2019-03-17,2019-03-17,2019-03-29,CAD,RV,1.0,NaN,19996.62,2019-03-19,CA10,2.960536e+09,0,5.0
46584,U001,0200486270,BAR corp,2019-12-18,2019.0,1.930196e+09,2019-12-03,2019-12-02,2019-12-03,2019-12-18,USD,RV,1.0,NaN,2678.97,2019-12-03,NAA8,1.930196e+09,0,0.0
46585,U001,0200886415,COSTCO systems,2020-03-05,2020.0,1.930527e+09,2020-02-18,2020-02-14,2020-02-18,2020-03-04,USD,RV,1.0,NaN,17642.88,2020-02-18,NAA8,1.930527e+09,0,1.0


In [77]:
val['business_code'].value_counts()

U001    11963
CA02     1151
U013      171
U002       45
U005        2
U007        1
Name: business_code, dtype: int64

In [78]:
val['business_code'].value_counts()

U001    11963
CA02     1151
U013      171
U002       45
U005        2
U007        1
Name: business_code, dtype: int64

In [79]:
val[val['business_code']=='U007']

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,document type,posting_id,area_business,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen,Delay
39689,U007,0100024767,GREEN,2019-08-23,2019.0,1.929724e+09,2019-08-08,2019-08-08,2019-08-08,2019-09-12,USD,RV,1.0,NaN,641.04,2019-08-08,NAG2,1.929724e+09,0,-20.0


In [80]:
val=pd.get_dummies(val,columns=["business_code"])

In [81]:
val.shape

(13333, 25)

In [82]:
val['business_code_U002'].value_counts()

0    13288
1       45
Name: business_code_U002, dtype: int64

In [83]:
val

,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,...,cust_payment_terms,invoice_id,isOpen,Delay,business_code_CA02,business_code_U001,business_code_U002,business_code_U005,business_code_U007,business_code_U013
32610,0200769623,WAL-MAR,2020-02-28,2020.0,1.930517e+09,2020-02-12,2020-02-12,2020-02-12,2020-02-27,USD,...,NAH4,1.930517e+09,0,1.0,0,1,0,0,0,0
32611,0200769623,WAL-MAR in,2019-12-09,2019.0,1.930198e+09,2019-11-28,2019-11-27,2019-11-28,2019-12-13,USD,...,NAH4,1.930198e+09,0,-4.0,0,1,0,0,0,0
32612,0200210681,HUB,2019-09-11,2019.0,1.929774e+09,2019-08-23,2019-08-22,2019-08-23,2019-09-12,USD,...,NAD1,1.929774e+09,0,-1.0,0,1,0,0,0,0
32613,0200729942,SA llc,2019-03-18,2019.0,1.928857e+09,2019-02-25,2019-02-25,2019-02-25,2019-03-12,USD,...,NAA8,1.928857e+09,0,6.0,0,1,0,0,0,0
32615,0200726979,BJ'S in,2019-02-19,2019.0,1.928727e+09,2019-02-02,2019-02-02,2019-02-02,2019-02-17,USD,...,NAA8,1.928727e+09,0,2.0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46582,0200759878,SA,2019-06-04,2019.0,1.929343e+09,2019-05-18,2019-05-18,2019-05-18,2019-06-02,USD,...,NAH4,1.929343e+09,0,2.0,0,1,0,0,0,0
46583,0140104440,SO co,2019-04-03,2019.0,2.960536e+09,2019-03-17,2019-03-17,2019-03-17,2019-03-29,CAD,...,CA10,2.960536e+09,0,5.0,1,0,0,0,0,0
46584,0200486270,BAR corp,2019-12-18,2019.0,1.930196e+09,2019-12-03,2019-12-02,2019-12-03,2019-12-18,USD,...,NAA8,1.930196e+09,0,0.0,0,1,0,0,0,0
46585,0200886415,COSTCO systems,2020-03-05,2020.0,1.930527e+09,2020-02-18,2020-02-14,2020-02-18,2020-03-04,USD,...,NAA8,1.930527e+09,0,1.0,0,1,0,0,0,0


In [84]:
val['invoice_currency'].value_counts()

USD    12182
CAD     1151
Name: invoice_currency, dtype: int64

In [85]:
val['document_create_date.1.year']=val['document_create_date.1'].dt.year
val['document_create_date.1.month']=val['document_create_date.1'].dt.month
val['document_create_date.1.Day']=val['document_create_date.1'].dt.day

In [86]:
val['due_in_date.year']=val['due_in_date'].dt.year
val['due_in_date.month']=val['due_in_date'].dt.month
val['due_in_date.Day']=val['due_in_date'].dt.day

In [87]:
val.shape

(13333, 31)

In [88]:
val['invoice_currency'].value_counts()

USD    12182
CAD     1151
Name: invoice_currency, dtype: int64

In [89]:
val["invoice_currency"]=val["invoice_currency"].astype('category')

In [90]:
val["invoice_currency_labelencoded"]=val["invoice_currency"].cat.codes

In [91]:
val.shape

(13333, 32)

In [92]:
val["invoice_currency_labelencoded"].value_counts()

1    12182
0     1151
Name: invoice_currency_labelencoded, dtype: int64

In [93]:
val['cust_payment_terms'].value_counts()

NAA8    5607
NAH4    3680
CA10    1124
NAC6     487
NAAX     345
NAD1     278
NAG2     275
NAU5     239
NA32     233
NAGD     209
NAX2     170
NA10      93
NAVE      83
NAD5      79
NAVF      68
NAAW      53
NAWU      43
NAVQ      36
NAVR      29
NAWN      26
NAWP      22
NA3B      15
NAVM      12
CA30      11
NAUZ      11
NACB      10
NAD4      10
NABG      10
NA38      10
C106       6
NAVD       6
NAB1       5
NA31       5
NA84       4
NA3F       4
NA9X       3
NAUI       3
NAVL       3
BR56       2
NATU       2
B052       2
NA8Q       2
CAX2       2
CAB1       2
BR12       2
NATV       2
NATJ       2
NATL       1
C129       1
NAV2       1
NAV9       1
NAUP       1
NATZ       1
NAWM       1
NANC       1
Name: cust_payment_terms, dtype: int64

In [94]:
val_mean=val.groupby('cust_payment_terms').mean()

In [95]:
val_mean.shape

(55, 21)

In [96]:
val.shape

(13333, 32)

In [97]:
val['cust_payment_terms'].describe()

count     13333
unique       55
top        NAA8
freq       5607
Name: cust_payment_terms, dtype: object

In [98]:
val[['cust_payment_terms','Delay']]

,cust_payment_terms,Delay
32610,NAH4,1.0
32611,NAH4,-4.0
32612,NAD1,-1.0
32613,NAA8,6.0
32615,NAA8,2.0
...,...,...
46582,NAH4,2.0
46583,CA10,5.0
46584,NAA8,0.0
46585,NAA8,1.0


In [99]:
x1=val.groupby(['cust_payment_terms'])['Delay'].mean().to_dict()

In [100]:
x1

{'B052': -5.0,
 'BR12': 6.5,
 'BR56': 4.5,
 'C106': -1.3333333333333333,
 'C129': -14.0,
 'CA10': 5.8336298932384345,
 'CA30': -12.818181818181818,
 'CAB1': 2.5,
 'CAX2': 89.0,
 'NA10': 2.6881720430107525,
 'NA31': -5.6,
 'NA32': -3.570815450643777,
 'NA38': 0.8,
 'NA3B': -6.666666666666667,
 'NA3F': 2.0,
 'NA84': -4.25,
 'NA8Q': 19.0,
 'NA9X': -7.0,
 'NAA8': 1.2200820403067594,
 'NAAW': -16.30188679245283,
 'NAAX': 0.008695652173913044,
 'NAB1': 7.2,
 'NABG': -2.1,
 'NAC6': 0.7145790554414785,
 'NACB': 5.0,
 'NAD1': -0.14388489208633093,
 'NAD4': 4.6,
 'NAD5': 9.708860759493671,
 'NAG2': 6.4363636363636365,
 'NAGD': -13.430622009569378,
 'NAH4': -1.9293478260869565,
 'NANC': 3.0,
 'NATJ': 1.0,
 'NATL': 15.0,
 'NATU': 1.0,
 'NATV': 1.0,
 'NATZ': 3.0,
 'NAU5': 1.0711297071129706,
 'NAUI': 40.333333333333336,
 'NAUP': 1.0,
 'NAUZ': 6.2727272727272725,
 'NAV2': -74.0,
 'NAV9': 33.0,
 'NAVD': 4.666666666666667,
 'NAVE': 1.963855421686747,
 'NAVF': 15.911764705882353,
 'NAVL': -9.3333333333

In [101]:
val['cust_payment_terms_tencoded']=val['cust_payment_terms'].map(x1)

In [102]:
val.shape

(13333, 33)

In [103]:
val.head()

,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,...,business_code_U007,business_code_U013,document_create_date.1.year,document_create_date.1.month,document_create_date.1.Day,due_in_date.year,due_in_date.month,due_in_date.Day,invoice_currency_labelencoded,cust_payment_terms_tencoded
32610,0200769623,WAL-MAR,2020-02-28,2020.0,1.930517e+09,2020-02-12,2020-02-12,2020-02-12,2020-02-27,USD,...,0,0,2020,2,12,2020,2,27,1,-1.929348
32611,0200769623,WAL-MAR in,2019-12-09,2019.0,1.930198e+09,2019-11-28,2019-11-27,2019-11-28,2019-12-13,USD,...,0,0,2019,11,28,2019,12,13,1,-1.929348
32612,0200210681,HUB,2019-09-11,2019.0,1.929774e+09,2019-08-23,2019-08-22,2019-08-23,2019-09-12,USD,...,0,0,2019,8,23,2019,9,12,1,-0.143885
32613,0200729942,SA llc,2019-03-18,2019.0,1.928857e+09,2019-02-25,2019-02-25,2019-02-25,2019-03-12,USD,...,0,0,2019,2,25,2019,3,12,1,1.220082
32615,0200726979,BJ'S in,2019-02-19,2019.0,1.928727e+09,2019-02-02,2019-02-02,2019-02-02,2019-02-17,USD,...,0,0,2019,2,2,2019,2,17,1,1.220082


In [104]:
val.drop(['cust_number','name_customer','invoice_currency','document type','cust_payment_terms','clear_date','posting_date','document_create_date','document_create_date.1','due_in_date','baseline_create_date'],axis=1,inplace=True)

In [105]:
val.shape

(13333, 22)

In [106]:
constant_filter2=VarianceThreshold(threshold=0)
constant_filter2.fit(val)

VarianceThreshold(threshold=0)

In [107]:
constant_filter2.get_support().sum()

19

In [108]:
constant_columns = [column for column in val.columns
                    if column not in val.columns[constant_filter2.get_support()]]

print((constant_columns))

['posting_id', 'area_business', 'isOpen']


In [109]:
quasi_constant_filter2=VarianceThreshold(threshold=0.01)

In [110]:
quasi_constant_filter2.fit(val)

VarianceThreshold(threshold=0.01)

In [111]:
quasi_constant_filter2.get_support().sum()

16

In [112]:
constant_columns = [column for column in val.columns
                    if column not in val.columns[quasi_constant_filter2.get_support()]]

print((constant_columns))

['posting_id', 'area_business', 'isOpen', 'business_code_U002', 'business_code_U005', 'business_code_U007']


In [113]:
val.drop(['posting_id', 'area_business', 'isOpen', 'business_code_U002', 'business_code_U005', 'business_code_U007'],axis=1,inplace=True)

In [114]:
val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13333 entries, 32610 to 46586
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   buisness_year                  13333 non-null  float64
 1   doc_id                         13333 non-null  float64
 2   total_open_amount              13333 non-null  float64
 3   invoice_id                     13333 non-null  float64
 4   Delay                          13333 non-null  float64
 5   business_code_CA02             13333 non-null  uint8  
 6   business_code_U001             13333 non-null  uint8  
 7   business_code_U013             13333 non-null  uint8  
 8   document_create_date.1.year    13333 non-null  int64  
 9   document_create_date.1.month   13333 non-null  int64  
 10  document_create_date.1.Day     13333 non-null  int64  
 11  due_in_date.year               13333 non-null  int64  
 12  due_in_date.month              13333 non-n

In [115]:
X_val_T=val.T

In [116]:
X_val_T.shape

(16, 13333)

In [117]:
X_val_T=pd.DataFrame(X_val_T)

In [118]:
X_val_T.duplicated().sum()

1

In [119]:
duplicated_feature2=X_val_T.duplicated()
duplicated_feature2

buisness_year                    False
doc_id                           False
total_open_amount                False
invoice_id                        True
Delay                            False
business_code_CA02               False
business_code_U001               False
business_code_U013               False
document_create_date.1.year      False
document_create_date.1.month     False
document_create_date.1.Day       False
due_in_date.year                 False
due_in_date.month                False
due_in_date.Day                  False
invoice_currency_labelencoded    False
cust_payment_terms_tencoded      False
dtype: bool

In [120]:
features_to_keep2=[not index for index in duplicated_feature2]

In [121]:
X_val_unique=X_val_T[features_to_keep2].T

In [122]:
X_val_unique.shape

(13333, 15)

In [123]:
val=X_val_unique

In [124]:
val

,buisness_year,doc_id,total_open_amount,Delay,business_code_CA02,business_code_U001,business_code_U013,document_create_date.1.year,document_create_date.1.month,document_create_date.1.Day,due_in_date.year,due_in_date.month,due_in_date.Day,invoice_currency_labelencoded,cust_payment_terms_tencoded
32610,2020.0,1.930517e+09,6899.20,1.0,0.0,1.0,0.0,2020.0,2.0,12.0,2020.0,2.0,27.0,1.0,-1.929348
32611,2019.0,1.930198e+09,1897.50,-4.0,0.0,1.0,0.0,2019.0,11.0,28.0,2019.0,12.0,13.0,1.0,-1.929348
32612,2019.0,1.929774e+09,13479.51,-1.0,0.0,1.0,0.0,2019.0,8.0,23.0,2019.0,9.0,12.0,1.0,-0.143885
32613,2019.0,1.928857e+09,9508.41,6.0,0.0,1.0,0.0,2019.0,2.0,25.0,2019.0,3.0,12.0,1.0,1.220082
32615,2019.0,1.928727e+09,1831.09,2.0,0.0,1.0,0.0,2019.0,2.0,2.0,2019.0,2.0,17.0,1.0,1.220082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46582,2019.0,1.929343e+09,39747.01,2.0,0.0,1.0,0.0,2019.0,5.0,18.0,2019.0,6.0,2.0,1.0,-1.929348
46583,2019.0,2.960536e+09,19996.62,5.0,1.0,0.0,0.0,2019.0,3.0,17.0,2019.0,3.0,29.0,0.0,5.833630
46584,2019.0,1.930196e+09,2678.97,0.0,0.0,1.0,0.0,2019.0,12.0,3.0,2019.0,12.0,18.0,1.0,1.220082
46585,2020.0,1.930527e+09,17642.88,1.0,0.0,1.0,0.0,2020.0,2.0,18.0,2020.0,3.0,4.0,1.0,1.220082


In [125]:
val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13333 entries, 32610 to 46586
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   buisness_year                  13333 non-null  float64
 1   doc_id                         13333 non-null  float64
 2   total_open_amount              13333 non-null  float64
 3   Delay                          13333 non-null  float64
 4   business_code_CA02             13333 non-null  float64
 5   business_code_U001             13333 non-null  float64
 6   business_code_U013             13333 non-null  float64
 7   document_create_date.1.year    13333 non-null  float64
 8   document_create_date.1.month   13333 non-null  float64
 9   document_create_date.1.Day     13333 non-null  float64
 10  due_in_date.year               13333 non-null  float64
 11  due_in_date.month              13333 non-null  float64
 12  due_in_date.Day                13333 non-n

In [126]:
val.shape

(13333, 15)

In [127]:
val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13333 entries, 32610 to 46586
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   buisness_year                  13333 non-null  float64
 1   doc_id                         13333 non-null  float64
 2   total_open_amount              13333 non-null  float64
 3   Delay                          13333 non-null  float64
 4   business_code_CA02             13333 non-null  float64
 5   business_code_U001             13333 non-null  float64
 6   business_code_U013             13333 non-null  float64
 7   document_create_date.1.year    13333 non-null  float64
 8   document_create_date.1.month   13333 non-null  float64
 9   document_create_date.1.Day     13333 non-null  float64
 10  due_in_date.year               13333 non-null  float64
 11  due_in_date.month              13333 non-null  float64
 12  due_in_date.Day                13333 non-n

In [128]:
test_set

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,document type,posting_id,area_business,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen,Delay
46587,U001,0200772595,SAFEW associates,NaT,2020.0,1.930745e+09,2020-04-06,2020-04-04,2020-04-06,2020-04-21,USD,RV,1.0,NaN,55148.86,2020-04-06,NAA8,1.930745e+09,1,NaN
46588,U001,0200711029,WEGMAN co,NaT,2020.0,1.930619e+09,2020-03-07,2020-03-07,2020-03-07,2020-03-22,USD,RV,1.0,NaN,81042.37,2020-03-07,NAA8,1.930619e+09,1,NaN
46589,U001,0200769623,WAL-MAR corporation,NaT,2020.0,1.930712e+09,2020-03-29,2020-03-28,2020-03-29,2020-04-13,USD,RV,1.0,NaN,20734.45,2020-03-29,NAH4,1.930712e+09,1,NaN
46590,U001,0200759878,SA systems,NaT,2020.0,1.930780e+09,2020-04-13,2020-04-13,2020-04-13,2020-04-28,USD,RV,1.0,NaN,12177.42,2020-04-13,NAH4,1.930780e+09,1,NaN
46591,U001,0100036066,GROC in,NaT,2020.0,1.930687e+09,2020-03-23,2020-03-23,2020-03-23,2020-04-07,USD,RV,1.0,NaN,36623.39,2020-03-23,NAA8,1.930687e+09,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,U001,0200726979,BJ'S llc,NaT,2020.0,1.930628e+09,2020-03-10,2020-03-09,2020-03-10,2020-03-25,USD,RV,1.0,NaN,1268.69,2020-03-10,NAA8,1.930628e+09,1,NaN
49996,U001,0100031704,DELHAIZE foundation,NaT,2020.0,1.930621e+09,2020-03-09,2020-03-08,2020-03-09,2020-03-24,USD,RV,1.0,NaN,58973.63,2020-03-09,NAA8,1.930621e+09,1,NaN
49997,U001,200780383,MEIJ foundation,NaT,2020.0,1.930671e+09,2020-03-20,2020-03-19,2020-03-20,2020-04-04,USD,RV,1.0,NaN,110574.78,2020-03-20,NAA8,1.930671e+09,1,NaN
49998,U013,0140103983,FACE corp,NaT,2020.0,1.991841e+09,2020-04-06,2020-04-02,2020-04-06,2020-06-05,USD,RV,1.0,NaN,40409.00,2020-04-06,NAUZ,1.991841e+09,1,NaN


In [129]:
test_set.sort_values(by='document_create_date.1',ascending=True).shape

(3413, 20)

In [130]:
test_set.nunique()

business_code                4
cust_number                436
name_customer             1027
clear_date                   0
buisness_year                1
doc_id                    3413
posting_date                77
document_create_date        85
document_create_date.1      77
due_in_date                114
invoice_currency             2
document type                1
posting_id                   1
area_business                0
total_open_amount         3273
baseline_create_date        78
cust_payment_terms          35
invoice_id                3413
isOpen                       1
Delay                        0
dtype: int64

In [131]:
test_set=test_set[test_set['document_create_date.1']<=test_set['posting_date']]

In [132]:
test_set.shape

(3413, 20)

In [133]:
test_set=test_set[test_set['document_create_date.1']<=test_set['baseline_create_date']]
test_set=test_set[test_set['document_create_date.1']<=test_set['due_in_date']]
#test_set=test_set[test_set['document_create_date.1']<=test_set['clear_date']]
test_set=test_set[test_set['posting_date']<=test_set['baseline_create_date']]
test_set=test_set[test_set['posting_date']<=test_set['due_in_date']]
#test_set=test_set[test_set['posting_date']<=test_set['clear_date']]
test_set=test_set[test_set['baseline_create_date']<=test_set['due_in_date']]

In [134]:
test_set.shape

(3258, 20)

In [135]:
test_set.dtypes

business_code                     object
cust_number                       object
name_customer                     object
clear_date                datetime64[ns]
buisness_year                    float64
doc_id                           float64
posting_date              datetime64[ns]
document_create_date      datetime64[ns]
document_create_date.1    datetime64[ns]
due_in_date               datetime64[ns]
invoice_currency                  object
document type                     object
posting_id                       float64
area_business                    float64
total_open_amount                float64
baseline_create_date      datetime64[ns]
cust_payment_terms                object
invoice_id                       float64
isOpen                             int64
Delay                            float64
dtype: object

In [136]:
#test_set[test_set.isnull().any(axis=1)]

In [137]:
test_set

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,document type,posting_id,area_business,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen,Delay
46587,U001,0200772595,SAFEW associates,NaT,2020.0,1.930745e+09,2020-04-06,2020-04-04,2020-04-06,2020-04-21,USD,RV,1.0,NaN,55148.86,2020-04-06,NAA8,1.930745e+09,1,NaN
46588,U001,0200711029,WEGMAN co,NaT,2020.0,1.930619e+09,2020-03-07,2020-03-07,2020-03-07,2020-03-22,USD,RV,1.0,NaN,81042.37,2020-03-07,NAA8,1.930619e+09,1,NaN
46589,U001,0200769623,WAL-MAR corporation,NaT,2020.0,1.930712e+09,2020-03-29,2020-03-28,2020-03-29,2020-04-13,USD,RV,1.0,NaN,20734.45,2020-03-29,NAH4,1.930712e+09,1,NaN
46590,U001,0200759878,SA systems,NaT,2020.0,1.930780e+09,2020-04-13,2020-04-13,2020-04-13,2020-04-28,USD,RV,1.0,NaN,12177.42,2020-04-13,NAH4,1.930780e+09,1,NaN
46591,U001,0100036066,GROC in,NaT,2020.0,1.930687e+09,2020-03-23,2020-03-23,2020-03-23,2020-04-07,USD,RV,1.0,NaN,36623.39,2020-03-23,NAA8,1.930687e+09,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,U001,0200726979,BJ'S llc,NaT,2020.0,1.930628e+09,2020-03-10,2020-03-09,2020-03-10,2020-03-25,USD,RV,1.0,NaN,1268.69,2020-03-10,NAA8,1.930628e+09,1,NaN
49996,U001,0100031704,DELHAIZE foundation,NaT,2020.0,1.930621e+09,2020-03-09,2020-03-08,2020-03-09,2020-03-24,USD,RV,1.0,NaN,58973.63,2020-03-09,NAA8,1.930621e+09,1,NaN
49997,U001,200780383,MEIJ foundation,NaT,2020.0,1.930671e+09,2020-03-20,2020-03-19,2020-03-20,2020-04-04,USD,RV,1.0,NaN,110574.78,2020-03-20,NAA8,1.930671e+09,1,NaN
49998,U013,0140103983,FACE corp,NaT,2020.0,1.991841e+09,2020-04-06,2020-04-02,2020-04-06,2020-06-05,USD,RV,1.0,NaN,40409.00,2020-04-06,NAUZ,1.991841e+09,1,NaN


In [138]:
test_set[test_set['business_code']=='U007']

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,document type,posting_id,area_business,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen,Delay


In [139]:
test_set=pd.get_dummies(test_set,columns=["business_code"])

In [140]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3258 entries, 46587 to 49999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   cust_number             3258 non-null   object        
 1   name_customer           3258 non-null   object        
 2   clear_date              0 non-null      datetime64[ns]
 3   buisness_year           3258 non-null   float64       
 4   doc_id                  3258 non-null   float64       
 5   posting_date            3258 non-null   datetime64[ns]
 6   document_create_date    3258 non-null   datetime64[ns]
 7   document_create_date.1  3258 non-null   datetime64[ns]
 8   due_in_date             3258 non-null   datetime64[ns]
 9   invoice_currency        3258 non-null   object        
 10  document type           3258 non-null   object        
 11  posting_id              3258 non-null   float64       
 12  area_business           0 non-null      flo

In [141]:
test_set.shape

(3258, 23)

In [142]:
test_set['business_code_U002'].value_counts()

0    3251
1       7
Name: business_code_U002, dtype: int64

In [143]:
test_set

,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,...,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen,Delay,business_code_CA02,business_code_U001,business_code_U002,business_code_U013
46587,0200772595,SAFEW associates,NaT,2020.0,1.930745e+09,2020-04-06,2020-04-04,2020-04-06,2020-04-21,USD,...,55148.86,2020-04-06,NAA8,1.930745e+09,1,NaN,0,1,0,0
46588,0200711029,WEGMAN co,NaT,2020.0,1.930619e+09,2020-03-07,2020-03-07,2020-03-07,2020-03-22,USD,...,81042.37,2020-03-07,NAA8,1.930619e+09,1,NaN,0,1,0,0
46589,0200769623,WAL-MAR corporation,NaT,2020.0,1.930712e+09,2020-03-29,2020-03-28,2020-03-29,2020-04-13,USD,...,20734.45,2020-03-29,NAH4,1.930712e+09,1,NaN,0,1,0,0
46590,0200759878,SA systems,NaT,2020.0,1.930780e+09,2020-04-13,2020-04-13,2020-04-13,2020-04-28,USD,...,12177.42,2020-04-13,NAH4,1.930780e+09,1,NaN,0,1,0,0
46591,0100036066,GROC in,NaT,2020.0,1.930687e+09,2020-03-23,2020-03-23,2020-03-23,2020-04-07,USD,...,36623.39,2020-03-23,NAA8,1.930687e+09,1,NaN,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0200726979,BJ'S llc,NaT,2020.0,1.930628e+09,2020-03-10,2020-03-09,2020-03-10,2020-03-25,USD,...,1268.69,2020-03-10,NAA8,1.930628e+09,1,NaN,0,1,0,0
49996,0100031704,DELHAIZE foundation,NaT,2020.0,1.930621e+09,2020-03-09,2020-03-08,2020-03-09,2020-03-24,USD,...,58973.63,2020-03-09,NAA8,1.930621e+09,1,NaN,0,1,0,0
49997,200780383,MEIJ foundation,NaT,2020.0,1.930671e+09,2020-03-20,2020-03-19,2020-03-20,2020-04-04,USD,...,110574.78,2020-03-20,NAA8,1.930671e+09,1,NaN,0,1,0,0
49998,0140103983,FACE corp,NaT,2020.0,1.991841e+09,2020-04-06,2020-04-02,2020-04-06,2020-06-05,USD,...,40409.00,2020-04-06,NAUZ,1.991841e+09,1,NaN,0,0,0,1


In [144]:
test_set['invoice_currency'].value_counts()

USD    2996
CAD     262
Name: invoice_currency, dtype: int64

In [145]:
test_set['document_create_date.1.year']=test_set['document_create_date.1'].dt.year
test_set['document_create_date.1.month']=test_set['document_create_date.1'].dt.month
test_set['document_create_date.1.Day']=test_set['document_create_date.1'].dt.day

In [146]:
test_set['due_in_date.year']=test_set['due_in_date'].dt.year
test_set['due_in_date.month']=test_set['due_in_date'].dt.month
test_set['due_in_date.Day']=test_set['due_in_date'].dt.day

In [147]:
test_set.shape

(3258, 29)

In [148]:
test_set['invoice_currency'].value_counts()

USD    2996
CAD     262
Name: invoice_currency, dtype: int64

In [149]:
test_set["invoice_currency"]=test_set["invoice_currency"].astype('category')

In [150]:
test_set["invoice_currency_labelencoded"]=test_set["invoice_currency"].cat.codes

In [151]:
test_set.shape

(3258, 30)

In [152]:
test_set["invoice_currency_labelencoded"].value_counts()

1    2996
0     262
Name: invoice_currency_labelencoded, dtype: int64

In [153]:
test_set['cust_payment_terms'].value_counts()

NAA8    1401
NAH4    1024
CA10     261
NAC6     138
NAGD     103
NAAX      81
NAU5      58
NAD1      31
NAG2      30
NA10      24
NA32      20
NAX2      18
NAD5      12
NAVE      10
NAAW       8
NAWP       7
NAUZ       5
NAVF       4
NAVQ       3
NAB1       3
NA38       3
C106       2
NAVD       2
NABG       2
NA84       2
NA3F       1
NACE       1
BR56       1
CA60       1
90M7       1
NACB       1
Name: cust_payment_terms, dtype: int64

In [154]:
test_set_mean=test_set.groupby('cust_payment_terms').mean()

In [155]:
test_set_mean.shape

(31, 19)

In [156]:
test_set.shape

(3258, 30)

In [157]:
test_set['cust_payment_terms'].describe()

count     3258
unique      31
top       NAA8
freq      1401
Name: cust_payment_terms, dtype: object

In [158]:
test_set[['cust_payment_terms','Delay']]

,cust_payment_terms,Delay
46587,NAA8,NaN
46588,NAA8,NaN
46589,NAH4,NaN
46590,NAH4,NaN
46591,NAA8,NaN
...,...,...
49995,NAA8,NaN
49996,NAA8,NaN
49997,NAA8,NaN
49998,NAUZ,NaN


In [159]:
x2=test_set.groupby(['cust_payment_terms'])['Delay'].mean().to_dict()

In [160]:
#x2

In [161]:
test_set['cust_payment_terms_tencoded']=test_set['cust_payment_terms'].map(x2)

In [162]:
test_set.shape

(3258, 31)

In [163]:
test_set.head()

,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,...,business_code_U002,business_code_U013,document_create_date.1.year,document_create_date.1.month,document_create_date.1.Day,due_in_date.year,due_in_date.month,due_in_date.Day,invoice_currency_labelencoded,cust_payment_terms_tencoded
46587,0200772595,SAFEW associates,NaT,2020.0,1.930745e+09,2020-04-06,2020-04-04,2020-04-06,2020-04-21,USD,...,0,0,2020,4,6,2020,4,21,1,NaN
46588,0200711029,WEGMAN co,NaT,2020.0,1.930619e+09,2020-03-07,2020-03-07,2020-03-07,2020-03-22,USD,...,0,0,2020,3,7,2020,3,22,1,NaN
46589,0200769623,WAL-MAR corporation,NaT,2020.0,1.930712e+09,2020-03-29,2020-03-28,2020-03-29,2020-04-13,USD,...,0,0,2020,3,29,2020,4,13,1,NaN
46590,0200759878,SA systems,NaT,2020.0,1.930780e+09,2020-04-13,2020-04-13,2020-04-13,2020-04-28,USD,...,0,0,2020,4,13,2020,4,28,1,NaN
46591,0100036066,GROC in,NaT,2020.0,1.930687e+09,2020-03-23,2020-03-23,2020-03-23,2020-04-07,USD,...,0,0,2020,3,23,2020,4,7,1,NaN


In [164]:
test_set.drop(['cust_number','name_customer','invoice_currency','document type','cust_payment_terms','clear_date','posting_date','document_create_date','document_create_date.1','due_in_date','baseline_create_date'],axis=1,inplace=True)

In [165]:
test_set.shape

(3258, 20)

In [166]:
constant_filter3=VarianceThreshold(threshold=0)
constant_filter3.fit(test_set)

VarianceThreshold(threshold=0)

In [167]:
constant_filter3.get_support().sum()

12

In [168]:
constant_columns = [column for column in test_set.columns
                    if column not in test_set.columns[constant_filter3.get_support()]]

print((constant_columns))

['buisness_year', 'posting_id', 'area_business', 'isOpen', 'Delay', 'document_create_date.1.year', 'due_in_date.year', 'cust_payment_terms_tencoded']


In [169]:
quasi_constant_filter3=VarianceThreshold(threshold=0.01)

In [170]:
quasi_constant_filter3.fit(test_set)

VarianceThreshold(threshold=0.01)

In [171]:
quasi_constant_filter3.get_support().sum()

10

In [172]:
constant_columns = [column for column in test_set.columns
                    if column not in test_set.columns[quasi_constant_filter3.get_support()]]

print((constant_columns))

['buisness_year', 'posting_id', 'area_business', 'isOpen', 'Delay', 'business_code_U002', 'business_code_U013', 'document_create_date.1.year', 'due_in_date.year', 'cust_payment_terms_tencoded']


In [173]:
test_set.drop(['buisness_year', 'posting_id', 'area_business', 'isOpen', 'Delay', 'business_code_U002', 'business_code_U013', 'document_create_date.1.year', 'due_in_date.year', 'cust_payment_terms_tencoded'],axis=1,inplace=True)

In [174]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3258 entries, 46587 to 49999
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   doc_id                         3258 non-null   float64
 1   total_open_amount              3258 non-null   float64
 2   invoice_id                     3258 non-null   float64
 3   business_code_CA02             3258 non-null   uint8  
 4   business_code_U001             3258 non-null   uint8  
 5   document_create_date.1.month   3258 non-null   int64  
 6   document_create_date.1.Day     3258 non-null   int64  
 7   due_in_date.month              3258 non-null   int64  
 8   due_in_date.Day                3258 non-null   int64  
 9   invoice_currency_labelencoded  3258 non-null   int8   
dtypes: float64(3), int64(4), int8(1), uint8(2)
memory usage: 213.2 KB


In [175]:
test_set.shape

(3258, 10)

In [176]:
test_set.info()#.shape,val.shape,train_data1.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3258 entries, 46587 to 49999
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   doc_id                         3258 non-null   float64
 1   total_open_amount              3258 non-null   float64
 2   invoice_id                     3258 non-null   float64
 3   business_code_CA02             3258 non-null   uint8  
 4   business_code_U001             3258 non-null   uint8  
 5   document_create_date.1.month   3258 non-null   int64  
 6   document_create_date.1.Day     3258 non-null   int64  
 7   due_in_date.month              3258 non-null   int64  
 8   due_in_date.Day                3258 non-null   int64  
 9   invoice_currency_labelencoded  3258 non-null   int8   
dtypes: float64(3), int64(4), int8(1), uint8(2)
memory usage: 213.2 KB


In [177]:
val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13333 entries, 32610 to 46586
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   buisness_year                  13333 non-null  float64
 1   doc_id                         13333 non-null  float64
 2   total_open_amount              13333 non-null  float64
 3   Delay                          13333 non-null  float64
 4   business_code_CA02             13333 non-null  float64
 5   business_code_U001             13333 non-null  float64
 6   business_code_U013             13333 non-null  float64
 7   document_create_date.1.year    13333 non-null  float64
 8   document_create_date.1.month   13333 non-null  float64
 9   document_create_date.1.Day     13333 non-null  float64
 10  due_in_date.year               13333 non-null  float64
 11  due_in_date.month              13333 non-null  float64
 12  due_in_date.Day                13333 non-n

In [178]:
train_data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31193 entries, 0 to 32609
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   buisness_year                  31193 non-null  float64
 1   doc_id                         31193 non-null  float64
 2   total_open_amount              31193 non-null  float64
 3   Delay                          31193 non-null  float64
 4   business_code_CA02             31193 non-null  float64
 5   business_code_U001             31193 non-null  float64
 6   business_code_U013             31193 non-null  float64
 7   document_create_date.1.year    31193 non-null  float64
 8   document_create_date.1.month   31193 non-null  float64
 9   document_create_date.1.Day     31193 non-null  float64
 10  due_in_date.year               31193 non-null  float64
 11  due_in_date.month              31193 non-null  float64
 12  due_in_date.Day                31193 non-null 

In [179]:
val.drop(['buisness_year', 'posting_id', 'area_business', 'isOpen', 'Delay', 'business_code_U002', 'business_code_U013', 'document_create_date.1.year', 'due_in_date.year', 'cust_payment_terms_tencoded'],axis=1,inplace=True)

KeyError: "['posting_id' 'area_business' 'isOpen' 'business_code_U002'] not found in axis"

In [ ]:
val.shape

In [ ]:
validate1,validate2=train_test_split(val,test_size=0.5,shuffle=False,stratify=None)

In [ ]:
validate1.shape

In [ ]:
validate1.shape,validate2.shape

In [ ]:
y_train=train_data1[['Delay']]
y_train

In [ ]:
validate1.info()

In [ ]:
y_validate1=validate1[['Delay']]
y_validate1


In [ ]:
y_validate2=validate2[['Delay']]
y_validate2

In [ ]:
#train_data1.drop(['Delay','cust_payment_terms_tencoded'],axis=1,inplace=True)
validate1.drop(['Delay','cust_payment_terms_tencoded'],axis=1,inplace=True)
validate2.drop(['Delay','cust_payment_terms_tencoded'],axis=1,inplace=True)

In [ ]:
train_data1.drop(['buisness_year','doc_id','total_open_amount','Delay','cust_payment_terms_tencoded'],axis=1,inplace=True)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
MSE_Score = []
R2_Score = []
Algorithm = []

In [ ]:
Algorithm.append('Linear Regression')
clf=LinearRegression()
clf.fit(train_data1,y_train)

# Predicting the Test Set Results
predicted=clf.predict(train_data1)

MSE_Score.append(mean_squared_error(y_train,predicted))
R2_Score.append(r2_score(y_train,predicted))


In [ ]:
Algorithm.append('Linear Regression')
clf=LinearRegression()
clf.fit(train_data1,y_train)

# Predicting the Test Set Results
predicted = clf.predict(validate1)

MSE_Score.append(mean_squared_error(y_validate1,predicted))
R2_Score.append(r2_score(y_validate1,predicted))



In [ ]:


Algorithm.append('Linear Regression')
clf = LinearRegression()
clf.fit(train_data1,y_train)

# Predicting the Test Set Results
predicted = clf.predict(validate2)

MSE_Score.append(mean_squared_error(y_validate2, predicted))
R2_Score.append(r2_score(y_validate2, predicted))


In [ ]:
Comparison = pd.DataFrame(list(zip(Algorithm, MSE_Score, R2_Score)), columns = ['Algorithm', 'MSE_Score', 'R2_Score'])
Comparison

In [ ]:
Algorithm.append('Linear Regression')
clf = LinearRegression()
clf.fit(train_data1,y_train)

# Predicting the Test Set Results
predicted = clf.predict(test_set)
predicted


In [ ]:
test_set.info()

In [ ]:
train_data1.info()

In [ ]:
val.info()

In [ ]:
import math
Y_test = pd.DataFrame(predicted,columns=['pred_delay'])
Y_test['pred_delay'] = Y_test.apply(lambda x : math.ceil(x),axis=1)
Y_test


In [ ]:
test_set['Delay']=Y_test['pred_delay'].values

In [ ]:
test_set['Delay']

In [ ]:
test_set

In [ ]:
from datetime import datetime, timedelta

In [ ]:
test_set['clear_date'] = test_set.apply(lambda x: x['due_in_date.Day'] + x['Delay'], axis=1)

In [ ]:
def bucketization(Delay):
    if Delay <= 15:
        bucket = '0-15 days'
    elif Delay in range(16,31):
        bucket = '16-30 days'
    elif Delay in range(31,46):
        bucket = '31-45 days'
    elif Delay in range(46,61):
        bucket = '46-60 days'
    else:
        bucket = 'Greater than 60'
    return bucket

In [ ]:
test_set['Aging_Bucket'] = test_set['Delay'].apply(bucketization)

In [ ]:
test_set['Delay'].max()

In [ ]:
test_set['predicted_date']=test_set['Delay']+test_set['due_in_date.Day']

In [ ]:
test_set['predicted_date'].max()

In [ ]:
test_set.head()

In [ ]:
#PREDICTED DATE OF INVOICE PAYMENT
test_set['predicted_date']